# Задание по лекциям 2.

- Не нужно использовать эффективные вычислительные алгоритмы. Нужно использовать те алгоритмы, что были представлены на лекции или практических занятиях. 
- Остальное как обычно: за "похожие" решения всем задействованным 0 баллов; если используете решение из открытого источника — обязательно вставьте ссылку; не удаляйте формулировки; не выкладывайте в открытый источник.
- Можно использовать `numpy.array` для матриц и матричной арифметики и `numpy.linalg` для подсчёта ранга и определителя, для вычисления обратной матрицы, решения СЛУ и т.п. То есть то, что вы уже реализовывали в прошлом семестре, ещё раз реализовывать необязательно. Более того, можно использовать в любом из *заданий по лекциям* функции, реализованные ранее в других *заданиях по лекциям*. Если возникнут сомнения, можно ли использовать ту или иную функцию — лучше сразу поинтересуйтесь у меня.

In [1]:
import numpy as np
import scipy.optimize
import scipy.linalg

In [2]:
# https://stackoverflow.com/questions/15638650/is-there-a-standard-solution-for-gauss-elimination-in-python
def gauss(A):
    pl, u = scipy.linalg.lu(A, permute_l=True)
    return scipy.optimize._remove_redundancy._remove_redundancy(u, np.zeros_like(u[:, 0]))[0]

In [33]:
def make_good(v):
    for i, e in enumerate(v):
        if abs(e) > 1e-10:
            return v / e

def make_world_better(A):
    new = []
    for i in range(A.shape[0]):
        new.append(make_good(A[i]))
    return np.array(new)

def fsr(A):
    bad_fsr = scipy.linalg.null_space(A).T
    return bad_fsr

$\mathbb{R}^n$ — вещественнозначное пространство вектор-**столбцов** со стандартным скалярным произведением.

**(1 балл) Задание 5.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ вектор-столбцов, и выдающую базис $$v = (v_1,\ldots,v_m),\ m \ge n-k,$$
ортогонального дополнения  для линейной оболочки данных векторов.

In [34]:
def orth_basis(U):
    # применяем гаусса к векторам U, чтобы выбрать из них линейно независимые
    U = gauss(U)
    # решим систему уравнений (u_1, x) = 0, ..., (u_k, x) = 0 (найдем фср)
    V = fsr(U)
    # вектора x_1, ..., x_p - фср - векторы базиса искомого орт.дополнения
    return V

In [35]:
U = np.array([
    [1, -2, 3, 1],
    [2, 1, 0, -1],
    [3, -1, 3, 0]
])

make_world_better(gauss(orth_basis(U)))

array([[ 1.        , -1.59404913, -1.53134971,  0.40595087],
       [ 0.        ,  1.        ,  0.33333333,  1.        ]])

**(1 балл) Задание 6.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ вектор-столбцов, и выдающую ортонормированный базис $$v = (v_1,\ldots,v_m),\ m \ge n-k,$$
для линейной оболочки данных векторов.

In [36]:
def gs(X):
    Q, _ = np.linalg.qr(X) # Как сказали Владу, это можно использовать
    return Q.T

In [38]:
make_world_better(gs(np.array([
    [1,1],
    [1,0]
])))

array([[ 1.,  1.],
       [ 1., -1.]])

**(1 балл) Задание 7.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ линейно независимых вектор-столбцов, и выдающую ортонормированный базис $$v = (v_1,\ldots,v_n)$$
всего пространства, такой что линейная оболочка векторов $(v_1,\ldots,v_k)$ совпадает с линейной оболочкой векторов $(u_1,\ldots,u_k)$.